# PPO
---

### 1. Import the Necessary Packages

In [1]:
from model.ppo_2 import PPO
from env.wrappers import LunarContinuous, LunarLanderWithUnknownWind,LunarLanderWithKnownWind
from utils.logger import WandbSummaryWritter

### 2. Instantiate the Model

Setup the hyperparameters in the code cell below.

In [2]:
# hyperparameters = {
#     'timesteps_per_batch': 1024 ,                # Number of timesteps to run per batch
#     'max_timesteps_per_episode': 1200,           # Max number of timesteps per episode
#     'n_updates_per_iteration': 5,                # Number of times to update actor/critic per iteration
#     'lr': 2.5e-4 ,                                # Learning rate of actor optimizer
#     'gamma': 0.95,                               # Discount factor to be applied when calculating Rewards-To-Go
# }
# hyperparameters = {'gamma': 0.999, 'lr_gamma': 0.995,
#                    'max_timesteps_per_episode': 1200,'lr': 0.005 }

hyperparameters = {}

misc_hyperparameters = {
    'env': LunarLanderWithUnknownWind
}

Initialise wandb session in the code cell below.

In [3]:
LOG = False
if LOG:
    logger = WandbSummaryWritter(project='lunar', config =misc_hyperparameters['env']().load_hyperparameters())
else:
    logger=None

Initialise the model fo the desired timestamps. Alternatively can specify a checkpoint to continue training.

In [4]:
checkpoint = 'ppo_checkpoints/non_wandb/ppo_policy_50.pth'
LOAD_MODEL = False

ppo = PPO(logger, **hyperparameters, **misc_hyperparameters)

if LOAD_MODEL:
    ppo.restore_savestate(checkpoint)

### 3. Train the Model

Train model for specified amount of timestamps.

In [5]:
ppo.train()


-------------------- Iteration #1 --------------------
Simulated timesteps: 5000
Average Episodic Return: -181.14
Average Actor Loss: -0.21708
Average Critic Loss: 1197.5432688395183
Average KL Divergence: 0.004371957431285409
Iteration took: 4.13 secs, of which rollout took 3.72 secs and gradient updates took 0.41 secs
Current actor learning rate: 0.0025
Current critic learning rate: 0.005
------------------------------------------------------


-------------------- Iteration #2 --------------------
Simulated timesteps: 10000
Average Episodic Return: -141.8
Average Actor Loss: -0.21699
Average Critic Loss: 1253.7105652491252
Average KL Divergence: 0.0036049219499651977
Iteration took: 3.94 secs, of which rollout took 3.73 secs and gradient updates took 0.19 secs
Current actor learning rate: 0.0025
Current critic learning rate: 0.005
------------------------------------------------------


-------------------- Iteration #1 --------------------
Simulated timesteps: 12000
Average adp Lo

### 4. Evaluate the Model

Run multiple episodes from pretrained model.

In [6]:
# ppo.validate(val_iter=30)
# ppo.device = 'cuda'
ppo.validate(3)
# ppo.test()

(-137.84169587413973, 90.33333333333333)